In [1]:
from src.data_prep import prepare_dataset

# One-time setup (idempotent)
prepare_dataset()

[INFO] Preparing data...
[INFO] Data split complete.


In [2]:
from src.utils import get_device, seed_everything

# Reproducibility
seed_everything()

# Device
device = get_device()
print("Using device:", device)

Using device: cuda


In [3]:
from src.data_prep import get_dataloaders
train_loader, val_loader = get_dataloaders("../data/processed", batch_size=32)

# Peek at a batch
images, labels = next(iter(train_loader))
print("Image batch shape:", images.shape)
print("Label batch shape:", labels.shape)

Image batch shape: torch.Size([32, 3, 224, 224])
Label batch shape: torch.Size([32])


In [4]:
import os
from src.model import build_model
num_classes = len(os.listdir("../data/processed/train"))

model = build_model(num_classes).to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

C:\Users\phild\PycharmProjects\ocular-ai\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\phild\PycharmProjects\ocular-ai\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
from torch import nn, optim
from src.train import train_one_epoch

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 5

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    train_loss, train_acc = train_one_epoch(model, train_loader, device, criterion, optimizer)
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")


Epoch 1/5
Train Loss: 0.6838, Train Accuracy: 75.56%

Epoch 2/5
Train Loss: 0.3433, Train Accuracy: 87.67%

Epoch 3/5
Train Loss: 0.2551, Train Accuracy: 90.71%

Epoch 4/5
Train Loss: 0.2034, Train Accuracy: 92.10%

Epoch 5/5
Train Loss: 0.1695, Train Accuracy: 92.86%


In [9]:
from torch import nn, optim
from src.train import train_one_epoch
from src.train import validate_one_epoch

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 5

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    train_loss, train_acc = train_one_epoch(model, train_loader, device, criterion, optimizer)
    val_loss, val_acc = validate_one_epoch(model, val_loader, device, criterion)

    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
    print(f"Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")



Epoch 1/5
Train Loss: 0.6831, Accuracy: 75.70%
Val   Loss: 0.4512, Accuracy: 82.94%

Epoch 2/5
Train Loss: 0.3426, Accuracy: 87.31%
Val   Loss: 0.3938, Accuracy: 85.24%

Epoch 3/5
Train Loss: 0.2496, Accuracy: 90.88%
Val   Loss: 0.3893, Accuracy: 85.15%

Epoch 4/5
Train Loss: 0.2035, Accuracy: 92.16%
Val   Loss: 0.3881, Accuracy: 86.87%

Epoch 5/5
Train Loss: 0.1739, Accuracy: 92.85%
Val   Loss: 0.3846, Accuracy: 86.54%
